In [1]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import widgets, Output
import torch
from PIL import Image
from era_data import TabletPeriodDataset, get_IDS
from VAE_model_tablets_class import VAE
import pandas as pd
from visualization_funcs import generate_image_from_VAE
import plotly.graph_objs as go
from ipywidgets import Image, Layout, HBox, VBox, Label
from sklearn.cluster import KMeans
from PIL import Image as PILImage
import io
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# IMG_DIR = 'output/images'
RUN_NAME_SUFFIX = '-masked_w_classification_loss' # ''
IMG_DIR = 'output/images_preprocessed'
LR = 5e-5
EPOCHS = 30
BATCH_SIZE = 16
SUFFIX = '-resnet50'
DATE = 'Oct2-v3'

In [3]:
IDS = get_IDS(IMG_DIR=IMG_DIR)
len(IDS)

VERSION_NAME = f'period_clf_bs{BATCH_SIZE}_lr{LR}_{EPOCHS}epochs{SUFFIX}-{len(IDS)}_samples{RUN_NAME_SUFFIX}_blurvae-conv-{DATE}'

num_classes = len(TabletPeriodDataset.PERIOD_INDICES)

class_weights = torch.load("data/class_weights_period.pt")

In [4]:
chekpoint_path = f'lightning_logs/{VERSION_NAME}/checkpoints/epoch=29-step=407516.ckpt'

In [5]:
vae_model = VAE.load_from_checkpoint(chekpoint_path,image_channels=1,z_dim=16, lr =1e-5, use_classification_loss=True, num_classes=num_classes,
            loss_type="weighted", class_weights=class_weights, device = device)

/sise/mickyfi-group/kapond/AnalysisBySynthesis/VAE_model_tablets_class.py:95: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.class_weights = torch.tensor(class_weights).to(device)


In [6]:
df_encodings_train = pd.read_csv('vae_encodings_and_data/vae_encoding_df_Oct2-v3_w_class_train.csv')
df_encodings_test = pd.read_csv('vae_encodings_and_data/vae_encoding_df_Oct2-v3_w_class_test.csv')

In [7]:
df_encodings_train

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,Period,Genre,Period_Name,Genre_Name
0,0.695620,-0.326637,1.869451,1.120124,-3.230606,1.078223,-2.286818,-0.213600,-2.957027,0.395746,0.686147,2.156204,1.430726,-0.226425,0.980338,-1.268349,1,1,Ur III,Administrative
1,0.488771,-0.505401,2.806634,-0.011833,-3.098880,0.062783,-1.171744,0.740006,-3.654613,0.205812,1.805598,1.383725,0.076415,-0.259734,0.640475,-2.054135,1,1,Ur III,Administrative
2,-0.164733,-0.477150,0.875877,1.744388,-1.792805,1.181497,-1.963445,-1.465748,-3.112858,1.478121,0.527789,-0.672934,0.073764,-0.231725,1.831303,-1.136168,1,1,Ur III,Administrative
3,0.032329,0.733997,1.136432,0.980828,-0.611278,0.674751,-2.201123,-0.782052,-3.235670,-0.053801,2.460866,0.277234,1.209431,-0.855141,1.686832,-1.203823,1,1,Ur III,Administrative
4,-2.117302,0.836886,2.241112,-2.359340,-0.441417,0.800746,0.511654,-1.293421,-2.536816,-0.735655,1.384294,-0.431612,1.212609,2.165188,0.814998,-1.704666,1,1,Ur III,Administrative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95083,0.239176,-1.923309,-2.530270,-1.414568,-2.259939,3.442629,-1.488173,3.042659,-1.457338,2.377469,-1.447516,-0.033868,-5.858948,-0.102388,-0.006196,-2.442893,1,1,Ur III,Administrative
95084,1.455029,-2.804560,-1.519120,-0.844761,0.119485,4.548380,-3.053493,2.669134,-1.779485,3.313068,-4.855679,-0.840578,-1.970150,-2.371216,-2.190817,-2.478782,1,4,Ur III,Royal/Monumental
95085,-1.905614,2.131715,1.252587,-1.206944,-1.331858,-1.755407,0.406348,0.143557,1.187520,-2.829051,-0.125474,-1.720152,-1.169059,-2.028571,3.087566,-1.271153,3,9,Old Babylonian,School
95086,-0.179766,-0.862341,1.356933,-0.642790,0.994281,-0.645911,-0.318751,-1.104957,-0.800633,0.576582,-0.364576,1.692541,0.775255,-0.038733,4.185235,-1.850742,3,9,Old Babylonian,School


In [8]:
df_filtered = df_encodings_train[df_encodings_train['Period_Name'].isin(['Old Babylonian', 'Ur III', 'Neo-Assyrian'])]

In [9]:
X = df_filtered[df_filtered.filter(regex='X').columns]

In [10]:
set(df_filtered["Period_Name"])

{'Neo-Assyrian', 'Old Babylonian', 'Ur III'}

In [11]:
scaler = StandardScaler()
data_std = scaler.fit_transform(X)

# Apply PCA to reduce dimensions
pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_std)

# Apply K-Means clustering
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)

# The labels_ attribute holds the cluster labels for each sample
df_filtered['kmeans_labels'] = kmeans.labels_
cluster_centers = kmeans.cluster_centers_

/home/kapond/.conda/envs/dani_torch/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/tmp/ipykernel_3169/3542707594.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['kmeans_labels'] = kmeans.labels_


In [12]:
# Function to convert PyTorch tensor to image for display
def tensor_to_image(tensor):
    # Normalize to 0-255 and convert to uint8
    tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min()) * 255
    np_array = tensor.numpy().astype(np.uint8).transpose(1, 2, 0)  # Convert to HWC format for PIL
    return PILImage.fromarray(np_array)

In [13]:
import matplotlib.pyplot as plt
from ipywidgets import widgets, Layout, VBox, Label
import io
import numpy as np
import mplcursors
from PIL import Image

# Define your tensor_to_image function here
def tensor_to_image(tensor):
    # Normalize the tensor to the range [0, 255]
    tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min()) * 255
    # Convert to a numpy array and ensure type is uint8
    np_array = tensor.cpu().detach().numpy().astype(np.uint8)
    # If it's a grayscale image, we need to make sure it's 2D
    if np_array.ndim == 3 and np_array.shape[0] == 1:
        np_array = np_array.squeeze(0)
    return Image.fromarray(np_array)

# Create widgets for the image and label
info_label = Label(value='Hover over a point to display the image')
image_widget = widgets.Image(layout=Layout(height='200px', width='200px', border='2px solid black'))

# Create an output widget to host the plot
output_widget = widgets.Output()

with output_widget:
    # Generate the plot
    fig, ax = plt.subplots(figsize=(10, 10))
    scatter = ax.scatter(X.iloc[:, 2], X.iloc[:, 1], c=df_filtered['Period'], cmap='viridis', s=5, alpha = 0.3)
    ax.set_xlabel('Component 1')
    ax.set_ylabel('Component 2')
    plt.show()

# Function to update the image_widget and label
def update_image(index):
    # Retrieve the original embedding
    original_embedding = X[index]
    # Generate image tensor from VAE
    image_tensor = generate_image_from_VAE(original_embedding, vae_model)
    # Convert the tensor to an image
    pil_img = tensor_to_image(image_tensor)
    # Convert to byte array for the widget
    img_byte_arr = io.BytesIO()
    pil_img.save(img_byte_arr, format='PNG')
    # Update the widget value
    image_widget.value = img_byte_arr.getvalue()
    # Update the label value
    info_label.value = f'Selected point index: {index}'

# Add hover functionality
mplcursors.cursor(scatter).connect("add", lambda sel: update_image(sel.index))

# Display the widgets
vbox = VBox([info_label, output_widget, image_widget])
display(vbox)
